In [1]:
import numpy as np
from PIL import Image
import os
import random
from scipy.optimize import differential_evolution
from typing import List, Tuple, Callable, Any


In [2]:
def calculate_mse(img1: np.ndarray, img2: np.ndarray) -> float:
    """Возвращает среднеквадратичную ошибку между двумя изображениями."""
    return np.mean((img1 - img2) ** 2)

In [10]:
from typing import List

def load_images_from_folder(path: str) -> List[np.ndarray]:
    """
    Рекурсивно обходит каталог `path` и все его подпапки, 
    загружая все файлы изображений и возвращая список numpy-массивов.
    """
    imgs = []
    for root, _, files in os.walk(path):
        for fname in files:
            full_path = os.path.join(root, fname)
            try:
                image = Image.open(full_path).convert('RGB')
                imgs.append(np.array(image))
            except Exception as e:
                print(f"Пропущен файл {full_path}: не является изображением или ошибка загрузки ({e})")
    return imgs


In [11]:
class Optimizer:
    """Абстрактный базовый класс для оптимизаторов."""
    def __init__(self, name: str):
        self.name = name

    def optimize(self, func: Callable, bounds: List[Tuple[float, float]], **kwargs) -> Tuple[Any, float]:
        raise NotImplementedError("Метод optimize должен быть реализован в подклассе.")

class MonteCarloOptimizer(Optimizer):
    """Оптимизация методом Монте-Карло"""
    def __init__(self, iterations: int = 1000):
        super().__init__("monte_carlo")
        self.iterations = iterations

    def optimize(self, func: Callable, bounds: List[Tuple[float, float]], **kwargs) -> Tuple[List[float], float]:
        best_point = None
        best_value = float('inf')
        dim = len(bounds)
        for _ in range(self.iterations):
            # Генерируем случайную точку внутри границ
            point = [random.uniform(b[0], b[1]) for b in bounds]
            val = func(point)
            if val < best_value:
                best_value = val
                best_point = point
        return best_point, best_value

class DifferentialEvolutionOptimizer(Optimizer):
    """Оптимизация методом дифференциальной эволюции"""
    def __init__(self, maxiter: int = 100):
        super().__init__("differential_evolution")
        self.maxiter = maxiter

    def optimize(self, func: Callable, bounds: List[Tuple[float, float]], **kwargs) -> Tuple[np.ndarray, float]:
        result = differential_evolution(func, bounds, maxiter=self.maxiter, disp=False)
        return result.x, result.fun


In [12]:
class ResultManager:
    """Хранит результаты оптимизации в списке словарей."""
    def __init__(self):
        self.records = []

    def add(self, method_name: str, best_point: Any, best_metric: float) -> None:
        self.records.append({
            'method': method_name,
            'best_point': best_point,
            'best_mse': best_metric
        })

    def get_all(self) -> List[dict]:
        return self.records


In [13]:
def make_objective(reference: np.ndarray, target: np.ndarray) -> Callable:
    """Создаёт функцию, принимающую параметры и возвращающую MSE."""
    def objective(params: List[float]) -> float:
        transformed = target  # Без изменений
        return calculate_mse(reference, transformed)
    return objective


In [14]:
if __name__ == '__main__':
    folder = 'blod_cell'
    imgs = load_images_from_folder(folder)
    if not imgs:
        print('Папка пуста или нет изображений')
        raise SystemExit

    reference_img = imgs[0]

    bounds = [(0.5, 2.0),  
              (0.5, 2.0)] 

    manager = ResultManager()
    optimizers = [MonteCarloOptimizer(iterations=500), DifferentialEvolutionOptimizer(maxiter=50)]

    for opt in optimizers:
        print(f"\n--- Запуск метода: {opt.name} ---")
        obj_func = make_objective(reference_img, reference_img)
        best_pt, best_val = opt.optimize(obj_func, bounds)
        manager.add(opt.name, best_pt, best_val)
        print(f"Лучшие параметры: {best_pt}")
        print(f"Значение MSE: {best_val}")

    print("\n=== Сводка всех результатов ===")
    for rec in manager.get_all():
        print(f"Метод: {rec['method']}, Лучшая точка: {rec['best_point']}, MSE: {rec['best_mse']}")



--- Запуск метода: monte_carlo ---
Лучшие параметры: [1.8964413116022216, 1.2429097048190882]
Значение MSE: 0.0

--- Запуск метода: differential_evolution ---
Лучшие параметры: [1.26822734 0.56915435]
Значение MSE: 0.0

=== Сводка всех результатов ===
Метод: monte_carlo, Лучшая точка: [1.8964413116022216, 1.2429097048190882], MSE: 0.0
Метод: differential_evolution, Лучшая точка: [1.26822734 0.56915435], MSE: 0.0
